In [1]:
from T4MLTraining import *
from tensorflow.python.keras.models import clone_model
from copy import deepcopy

2021-08-13 12:04:50.175385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-13 12:04:50.175412: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class Client:
    def __init__(self, id_, inp_train, out_train, inp_test, out_test):
        self.id_ = id_
        self.inp_train = inp_train
        self.out_train = out_train
        self.inp_test = inp_test
        self.out_test = out_test
        self.deviation = 1

    def getWeight(self):
        return sum(flatten(self.inp_train.tolist()[0]))
        
    def extract(self):
        return self.inp_train, self.out_train, self.inp_test, self.out_test
    
    def setModel(self, model):
        model_copy = clone_model(model.model)
        #model_copy.build((None, len(self.inp_train[0]), 1))
        model_copy.compile(optimizer='adam', loss='mse')
        model_copy.set_weights(model.model.get_weights())
        model = deepcopy(model)
        model.model = model_copy
        self.model = model
        self.model.setData(self.inp_train, self.out_train)

    def train(self, draw_loss=False):
        ts = time.time()
        self.model.train()
        te = time.time()
        print("Cli# {0}: {1:.2f} sec".format(self.id_, te - ts), end = '; ')
        if draw_loss == True:
            self.model.drawLoss()

    def extractModel(self):
        return self.model
    
    def predict(self, frame_in, frame_out, verbose_=False):
        predictor = Prediction(self.inp_test,self.out_test,self.model, frame_in, frame_out)
        predictor.predict()
        predictor.summary("",verbose_)
        self.deviation = predictor.in_percent / 100
        return self.deviation

class Distributor:
    def __init__(self, dat, company, location, div_company=True, div_location=False, bin_='40T', frame_out=2, ratio=0.8):
        self.dat = dat
        self.company = company
        self.location = location
        self.divide_by_company = div_company
        self.divide_by_location = div_location # NOT YET IMPLEMENTED
        self.clients = []
        
        self.bin_=bin_
        self.frame_out = frame_out
        self.ratio = ratio

    def divByCompany(self):
        for c in self.company:
            tr = DataPrep(self.dat,[c],self.location,[c],self.location,self.bin_,self.frame_out,self.ratio)
            tr.setup()

            inp_train, out_train, inp_test, out_test = tr.extract()
            cli = Client(str(c), inp_train, out_train, inp_test, out_test)
            self.clients.append(cli)
    
    def extractClients(self):
        return self.clients


In [3]:
def weighting(model, wei):
    for i in range(len(model)):
        model[i] *= wei
    return model

def combine_models(m1, m2):
    m = deepcopy(m1)
    for i in range(len(m1)):
        m[i] += m2[i]
    return m

In [4]:
class Server:
    def __init__(self, clients, frame_in, frame_out, model_type, epochs_):
        self.clients = clients
        self.cli_models = []
        if model_type == 'CNNLSTM':
            self.model = CNNLSTM(frame_in, frame_out, epochs=epochs_)
        self.frame_in = frame_in
        self.frame_out = frame_out

    def askClients(self):
        for c in self.clients:
            c.train()
            m = c.extractModel()
            self.cli_models.append(m)      

    def aggregate(self, mode='amt'):
        multiplier = []
        cli_weights = []
        # get client demand size and weights
        for c in self.clients:
            if mode == 'amt':
                w = c.getWeight()
            elif mode == 'pre':
                w = c.predict(self.frame_in, self.frame_out, verbose_=0)
                try:
                    w = (1 / w)
                except ZeroDivisionError:
                    w = 1
            multiplier.append(w)
            cli_weights.append(c.model.model.get_weights())
        total_multiplier = sum(multiplier)

        # get proportions
        multiplier_proportion = []
        for w in multiplier:
            multiplier_proportion.append(w/total_multiplier)
        
        # averaging
        average = None
        for i in range(len(cli_weights)):
            weighted = weighting(cli_weights[i],multiplier_proportion[i])
            if average == None:
                average = weighted
            else:
                average = combine_models(average, weighted)

        # repacking in global model
        self.model.model.set_weights(average)
        self.model.model.build((None,self.frame_in,1))
        self.model.model.compile(optimizer='adam',loss='mse')

    def sendGlobalModel(self):
        for c in clients:
            c.setModel(self.model)

    def iterate(self, iters, mode='amt'):
        for i in range(iters):
            print("\nIteration " + str(i + 1) + " of " + str(iters), end=' | ')
            tsg = time.time()
            self.sendGlobalModel() # loop start
            tsh = time.time()
            self.askClients()
            tsi = time.time()
            self.aggregate(mode)
            tsj = time.time()
            print(" | Glob: {0:.2f}s, Train: {1:.2f}s, Aggr: {2:.2f}s.".format(tsh-tsg, tsi-tsh, tsj-tsi))
        return self

In [5]:
# Builder
dat = 'trips_simpler.csv'
bin_ = '40T'
company = [2,3,4,5]
location = [mk for mk in range(9)]
frame_in = binToWindows(bin_)
frame_out = 2
ratio = 0.8

In [6]:
# Distribute data to clients
db = Distributor(dat, company, location)
db.divByCompany()
clients = db.extractClients()

[2] [0, 1, 2, 3, 4, 5, 6, 7, 8] | Sum of all datalist: 596314 | Sum of all tested: 118692
[3] [0, 1, 2, 3, 4, 5, 6, 7, 8] | Sum of all datalist: 12732738 | Sum of all tested: 2738413
[4] [0, 1, 2, 3, 4, 5, 6, 7, 8] | Sum of all datalist: 851726 | Sum of all tested: 177922
[5] [0, 1, 2, 3, 4, 5, 6, 7, 8] | Sum of all datalist: 4201065 | Sum of all tested: 893440


In [7]:
# Server simulation
epochs = 2000
iterations = 1
serv = Server(clients, frame_in, frame_out, 'CNNLSTM', epochs_=int(epochs/iterations))
serv.iterate(iterations, mode='pre')

2021-08-13 12:05:57.477628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-13 12:05:57.477660: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-13 12:05:57.477679: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cl): /proc/driver/nvidia/version does not exist
2021-08-13 12:05:57.477849: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



Iteration 1 of 2 | 

2021-08-13 12:05:58.239754: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-13 12:05:58.240444: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2592000000 Hz


Cli# 2: 201.42 sec; Cli# 3: 207.38 sec; Cli# 4: 215.92 sec; Cli# 5: 211.55 sec;  | Glob: 0.56s, Train: 836.28s, Aggr: 27.95s.

Iteration 2 of 2 | Cli# 2: 208.62 sec; Cli# 3: 208.99 sec; Cli# 4: 1909.48 sec; Cli# 5: 160.48 sec;  | Glob: 0.52s, Train: 2487.57s, Aggr: 20.39s.


In [8]:
for c in serv.clients:
    c.predict(frame_in,frame_out, verbose_=True)

RMSE  = 45
Total demand  = [118692]
Average demand  = 567
Deviation  = [7.96]%
RMSE  = 574
Total demand  = [2738413]
Average demand  = 13102
Deviation  = [4.38]%
RMSE  = 44
Total demand  = [177922]
Average demand  = 851
Deviation  = [5.24]%
RMSE  = 229
Total demand  = [893440]
Average demand  = 4274
Deviation  = [5.37]%
